<a href="https://colab.research.google.com/github/ClovisL/FidelityOptionsTracker/blob/main/FidelityOptionsTracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Run all cells (**CTRL+F9**). A prompt will appear in this first cell to upload a file. Upload the **.csv** file containing transaction history, downloaded from Fidelity, ideally having chosen a 2 month time period containing all transaction dates of those months. The remaining cells will then be run, and a formatted Excel spreadsheet will be exported, and can be downloaded from the "Files" tab on the left, denoted by the 📁 **folder icon**. Click the folder icon to open the list of files, then right click and download the **formatted** report.

In [ ]:
from google.colab import files
# Brings up the prompt to upload a file
# If an error message appears, try enabling cookies for the site
data_to_load = files.upload()

In [ ]:
import pandas as pd
import numpy as np
import io
 
# Reads the .csv file and creates a data frame, skipping the first 5 rows that contain unneccessary information
report_df = pd.read_csv(list(data_to_load.keys())[0], skiprows=5)


# Creates a new dataframe with only the rows containing Puts and Calls
clean_df = report_df[report_df["Security Description"].str.contains("PUT|CALL", regex=True, case=False, na=False)]
#clean_df = report_df[report_df["Symbol"].str.contains("SPY", regex=True, case=False, na=False)]

# Drop index and unnecessary columns, rename columns to more descriptive names
clean_df = clean_df.drop("Account", axis=1)
clean_df = clean_df.drop("Action", axis=1)
clean_df = clean_df.drop("Symbol", axis=1)
clean_df = clean_df.drop("Security Type", axis=1)
clean_df = clean_df.drop("Price ($)", axis=1)
clean_df = clean_df.drop("Commission ($)", axis=1)
clean_df = clean_df.drop("Fees ($)", axis=1)
clean_df = clean_df.drop("Accrued Interest ($)", axis=1)
clean_df = clean_df.rename(columns={'Run Date': 'Transaction Date', 'Security Description': 'Action'})
clean_df = clean_df[["Transaction Date", "Action", "Quantity", "Amount ($)"]]
clean_df = clean_df.reset_index(drop=True)

# Splits the action column into multiple columns containing ticker, call/put, expiration date, and strike price
action_df = pd.DataFrame(clean_df["Action"].str.split().values.tolist())
clean_df = clean_df.drop("Action", axis=1)
clean_df["Ticker"] = action_df[1]
clean_df["Type"] = action_df[0]
# Reverses the order to grab relevant info
action_df = action_df.rename(columns={12: 0, 11:1, 10: 2, 9: 3, 8: 4, 7: 5, 5: 7, 4: 8, 3: 9, 2: 10, 1: 11, 0: 12})
# Loops over the columns, stopping once it reaches "SHS)" to skip over blanks, in order to find strike price and expiration date
c = 0
r = 0
for row in action_df.iterrows():
  # While the string hasn't been reached, the int representing column index increases, moving to the next column in the row to check
  while action_df[c][r] != "SHS)":
    c += 1
  # Once the string has been reached, the Expiration Date and Strike Price can be found and are added to the corresponding column/row in the data frame. Row index is increased, column is reset to 0
  # For date, the final 3 characters of the month string are taken, in case of any typos
  clean_df.loc[clean_df.index[r], "Expiration Date"] = action_df[c+5][r][-3::]+"-"+action_df[c+4][r]+"-"+action_df[c+3][r]
  clean_df.loc[clean_df.index[r], "Strike Price"] = action_df[c+2][r]
  r += 1
  c = 0

# Rearrange columns
clean_df = clean_df[["Transaction Date", "Ticker", "Type", "Expiration Date", "Strike Price", "Quantity", "Amount ($)"]]
# Changes Transaction and Expiration Date columns into a datetime format to be able to sort in order
clean_df["Transaction Date"] = pd.to_datetime(clean_df["Transaction Date"])
clean_df["Transaction Date"] = clean_df["Transaction Date"].dt.strftime('%m-%d-%Y')
clean_df["Expiration Date"] = pd.to_datetime(clean_df["Expiration Date"])
clean_df["Expiration Date"] = clean_df["Expiration Date"].dt.strftime('%m-%d-%Y')

clean_df

In [ ]:
# Adds up the total net P/L
Total = clean_df["Amount ($)"].sum().round(2)
print("Total P/L for the date range: ")
print(Total)

##Exports the dataframe as an Excel spreadsheet. After running this cell, a file will appear in the "Files" tab to the left, denoted by the 📁 folder icon. Click the folder icon to open the list of files, then right click and download the formatted report.

In [ ]:
import openpyxl

# Gets the month range for the file, to put in the file name for better reference
months = pd.to_datetime(clean_df["Transaction Date"])
months = months.dt.strftime('%b-%Y')
last_index = months.size - 1
month1 = months[last_index]
month2 = months[0]
# Takes the original file name, adds "Formatted" to the beginning, adds the month range, and exports the data frame as an Excel spreadsheet
file_name = next(iter(data_to_load))[:-4]
clean_df.to_excel(f'Formatted {file_name} {month1} to {month2}.xlsx', index = False)